In [81]:
import os
from glob import glob

import dask
import dask.bag as db
import dask.dataframe as dd
import dask.array as da
from distributed import Client
from dask_jobqueue import SLURMCluster
import pandas as pd 
import numpy as np

from IPython.display import display
import matplotlib.pyplot as plt

import ast
from tqdm import tqdm 
from functools import partial

In [58]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = False

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(memory='4GB', cores=2, python='/scratch/work/public/dask/bin/python', 
                               local_directory='/tmp/{}/'.format(os.environ['SLURM_JOB_USER']),
                               job_extra=['--output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])])

    cluster.submit_command = 'slurm'
    cluster.scale(1000)

    display(cluster)
    client = Client(cluster)

display(client)

/ext3/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37519 instead
  warnings.warn(


Client Scheduler: tcp://10.32.35.15:35215 Dashboard: http://10.32.35.15:37519/status,Cluster Workers: 6 Cores: 6 Memory: 12.00 GB


In [59]:
public_path = '/scratch/work/public/MillionSongDataset/AdditionalFiles/'
local_path = '/scratch/tj810/final-project-team_unsupervised_learners/feature_files/'
indexer_path = '/scratch/tj810/final-project-team_unsupervised_learners/model_outputs/id_mapping/'
target_path = '/scratch/tj810/final-project-team_unsupervised_learners/model_outputs/model_csv/MFImp_a1r100_reg0.1_it20/'

In [60]:
os.listdir(public_path)

['BROKEN_lastfm_tags.db',
 'README',
 'lastfm_tags.db',
 'LICENSE',
 'unique_terms.txt',
 'artist_term.db',
 'artist_location.txt',
 'artist_similarity.db',
 'mxm_779k_matches.txt',
 'sid_mismatches.csv',
 'rdio_ids.txt',
 'sid_mismatches.txt',
 'TRACKIDS.txt',
 'msd_summary_file.h5',
 'unique_tracks.txt',
 'mxm_dataset.db',
 'unique_mbtags.txt',
 'unique_artists.txt',
 'tracks_per_year.txt',
 'track_metadata.db']

In [5]:
# import unique terms 
unique_terms = list(pd.read_csv(public_path + 'unique_terms.txt',header = None)[0].values)
len(unique_terms)

7643

In [61]:
# flatten artist_terms to dummy variables
def get_dummy_terms(row):
    unique_terms = list(pd.read_csv('/scratch/work/public/MillionSongDataset/AdditionalFiles/unique_terms.txt',header = None)[0].values)
    terms = row['term']
    if pd.isna(terms):
        return [0] * len(unique_terms)
    else: 
        terms = ast.literal_eval(terms)
        return list(np.array([t in terms for t in unique_terms]) * 1)

Dask

In [63]:
feature_dd = dd.read_csv(local_path + 'merged_all_features.csv').repartition(1000)

In [64]:
# unique_terms = db.read_text(public_path + 'unique_terms.txt')
# def map_terms(terms,unique_terms):
#     return unique_terms.map(lambda x: x in terms)

In [65]:
meta = dd.utils.make_meta([(t, 'i8') for t in range(7643)])
terms = feature_dd.apply(get_dummy_terms,axis='columns', result_type='expand', meta = meta)

In [66]:
terms

Dask DataFrame Structure:
                   0      1      2      3      4      5      6      7      8      9      10     11     12     13     14     15     16     17     18     19     20     21     22     23     24     25     26     27     28     29     30     31     32     33     34     35     36     37     38     39     40     41     42     43     44     45     46     47     48     49     50     51     52     53     54     55     56     57     58     59     60     61     62     63     64     65     66     67     68     69     70     71     72     73     74     75     76     77     78     79     80     81     82     83     84     85     86     87     88     89     90     91     92     93     94     95     96     97     98     99     100    101    102    103    104    105    106    107    108    109    110    111    112    113    114    115    116    117    118    119    120    121    122    123    124    125    126    127    128    129    130    131    132    133    134    135    136    137    138    139    140    141    142    143    144    145    146    147    148    149    150    151    152    153    154    155    156    157    158    159    160    161    162    163    164    165    166    167    168    169    170    171    172    173    174    175    176    177    178    179    180    181    182    183    184    185    186    187    188    189    190    191    192    193    194    195    196    197    198    199    200    201    202    203    204    205    206    207    208    209    210    211    212    213    214    215    216    217    218    219    220    221    222    223    224    225    226    227    228    229    230    231    232    233    234    235    236    237    238    239    240    241    242    243    244    245    246    247    248    249    250    251    252    253    254    255    256    257    258    259    260    261    262    263    264    265    266    267    268    269    270    271    272    273    274    275    276    277    278    279    280    281    282    283    284    285    286    287    288    289    290    291    292    293    294    295    296    297    298    299    300    301    302    303    304    305    306    307    308    309    310    311    312    313    314    315    316    317    318    319    320    321    322    323    324    325    326    327    328    329    330    331    332    333    334    335    336    337    338    339    340    341    342    343    344    345    346    347    348    349    350    351    352    353    354    355    356    357    358    359    360    361    362    363    364    365    366    367    368    369    370    371    372    373    374    375    376    377    378    379    380    381    382    383    384    385    386    387    388    389    390    391    392    393    394    395    396    397    398    399    400    401    402    403    404    405    406    407    408    409    410    411    412    413    414    415    416    417    418    419    420    421    422    423    424    425    426    427    428    429    430    431    432    433    434    435    436    437    438    439    440    441    442    443    444    445    446    447    448    449    450    451    452    453    454    455    456    457    458    459    460    461    462    463    464    465    466    467    468    469    470    471    472    473    474    475    476    477    478    479    480    481    482    483    484    485    486    487    488    489    490    491    492    493    494    495    496    497    498    499    500    501    502    503    504    505    506    507    508    509    510    511    512    513    514    515    516    517    518    519    520    521    522    523    524    525    526    527    528    529    530    531    532    533    534    535    536    537    538    539    540    541    542    543    544    545    546    547    548    549    550    551    552    553    554    555    556    557    558    559    560    561    562    563    564    

In [86]:
terms = terms.compute()

KeyboardInterrupt: 

Second attempt

In [ ]:
terms.to_csv(local_path+'terms_dummies.csv')

In [87]:
# flatten artist_terms to dummy variables
def get_dummy_terms(section):
    dummies = []
    for row in section:
        unique_terms = list(pd.read_csv('/scratch/work/public/MillionSongDataset/AdditionalFiles/unique_terms.txt',header = None)[0].values)
        terms = row['term']
        if terms == da.nan:
            dummies.append([0] * len(unique_terms)) 
        else: 
            terms = ast.literal_eval(terms)
            dummies.append(list(da.array([t in terms for t in unique_terms]) * 1))
    return dummies

In [83]:
features = pd.read_csv(local_path + 'merged_all_features.csv',header = 0, index_col = 0)

/ext3/miniconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [88]:
meta = dd.utils.make_meta([(t, 'i8') for t in range(7643)])
features_dd = dd.from_pandas(features, npartitions=100)
terms = features_dd.map_partitions(get_dummy_terms, meta)

ValueError: Not all divisions are known, can't align partitions. Please use `set_index` to set the index.

Covert track_ids to indices using Spark's StringIndexer output